In [1]:
!pip install transformers

In [2]:
import json 
import tensorflow as tf
import os
import requests

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
if not os.path.exists('/content/drive/MyDrive/MRC'):
  os.mkdir('/content/drive/MyDrive/MRC')
  os.mkdir('/content/drive/MyDrive/MRC/squad')
  os.mkdir('/content/drive/MyDrive/MRC/aaqad')

In [5]:
#uncomment if you did not download the data
#url = 'https://rajpurkar.github.io/SQuAD-explorer/dataset/'
#for file in ['train-v2.0.json', 'dev-v2.0.json']:
#    res = requests.get(f'{url}{file}')
#    # write to file
#    with open(f'/content/drive/MyDrive/MRC/squad{file}', 'wb') as f:
#        for chunk in res.iter_content(chunk_size=4):
#            f.write(chunk)

In [6]:
#uncomment if you did not download the data
#!pip install gdown
#!gdown https://drive.google.com/uc?id=1jhUmWb9eHVATqhrWKAXxSE2gqJ53-wk6 -O AAQAD.json
#!gdown https://drive.google.com/uc?id=1V5ziIZe__pGg14nH42WyMEFz444XPWf7 -O AAQAD\-train.json
#!gdown https://drive.google.com/uc?id=19nj9jiCdJlHwAfgUTJ_Z8jg1cB34yfjv -O AAQAD\-dev.json
#!gdown https://drive.google.com/uc?id=1z0XksuTwnqhiX1guxkmjYmoNA_JZ6SUN -O AAQAD\-test.json



In [7]:
with open('/content/drive/MyDrive/MRC/aaqad/AAQAD-train.json') as f:
  aaqad_train_dict= json.load(f)

In [8]:
#Dictionay(data)--ListOfArticles--Dictionary(Paragraphs)--ListOfContext--ListOfQAS
print(aaqad_train_dict.keys())
print(aaqad_train_dict['data'][0].keys())
print(aaqad_train_dict['data'][0]['paragraphs'][0])#One Context Multiple qas(key)-contains list of qas
print(aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0])
question = aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0]['question']
id =aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0]['id']
answer =aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['text']
answer_start = aaqad_train_dict['data'][0]['paragraphs'][0]['qas'][0]['answers'][0]['answer_start']
print(f'Question: {question} it is id {id} Answer: {answer} Start Span: {answer_start}')

dict_keys(['version', 'data'])
dict_keys(['title', 'paragraphs'])
{'qas': [{'question': 'ما الذي تعتمد عليه بعض الطيور الكبيرة ذات الجناحين لمساعدتها على الارتفاع؟', 'id': 2728, 'answers': [{'text': 'تصل المسافة التي', 'answer_start': 211}], 'is_impossible': False}, {'question': 'متى تهاجر الطيور الجارحة الكبيرة؟', 'id': 2729, 'answers': [{'text': 'ليلا ونهارا،', 'answer_start': 433}], 'is_impossible': False}, {'question': 'ما مقدار وزن أجسامهم التي يفقدها طراد العسل أثناء الهجرة؟', 'id': 2730, 'answers': [{'text': 'إلى 50', 'answer_start': 245}], 'is_impossible': False}], 'context': 'هجرة الطيور هي رحلة موسمية تقوم بها أسراب من الطيور قاطعين مسافات هائلة عبر الصحاري وقمم الجبال العالية والمحيطات. تصل هذه الطيور إلى هدفها في وقت واحد يتطابق مع الوقت التي وصلت فيه في العام السابق. بعض الأنواع تصل المسافة التي تقطعها في هجرتها إلى 50 الف كيلومتر في السنة، البعض الآخر تستمر بالطيران بدون انقطاع لمدة تصل إلى 100 ساعة مع منظومة تحديد دقيقة للاتجاهات عند تلك الطيور. بعض الأنواع لها القدرة عل

In [9]:
def Read_AAQAD(path):
  contexts =[]
  answers =[]
  questions =[]
  with open(path) as f:
    aaqad_dict = json.load(f)
    for article in aaqad_dict['data']:#through each article
      for passage in article['paragraphs']:
        context = passage['context']
        for qa in passage['qas']:
          question = qa['question']
          if 'plausible_answers' in qa.keys():
            access = 'plausible_answers'
          else:
            access = 'answers'
          for answer in qa[access]:
            contexts.append(context)
            answers.append(answer)
            questions.append(question)
  return contexts,questions,answers

In [10]:
train_contexts, train_questions, train_answers = Read_AAQAD('/content/drive/MyDrive/MRC/aaqad/AAQAD-train.json')
val_contexts, val_questions, val_answers = Read_AAQAD('/content/drive/MyDrive/MRC/aaqad/AAQAD-dev.json')
test_contexts, test_questions, test_answers = Read_AAQAD('/content/drive/MyDrive/MRC/aaqad/AAQAD-test.json')

In [11]:
print(len(train_contexts))
print(len(val_contexts))
print(len(test_contexts))

12629
1926
3262


In [12]:
train_contexts[1]

'هجرة الطيور هي رحلة موسمية تقوم بها أسراب من الطيور قاطعين مسافات هائلة عبر الصحاري وقمم الجبال العالية والمحيطات. تصل هذه الطيور إلى هدفها في وقت واحد يتطابق مع الوقت التي وصلت فيه في العام السابق. بعض الأنواع تصل المسافة التي تقطعها في هجرتها إلى 50 الف كيلومتر في السنة، البعض الآخر تستمر بالطيران بدون انقطاع لمدة تصل إلى 100 ساعة مع منظومة تحديد دقيقة للاتجاهات عند تلك الطيور. بعض الأنواع لها القدرة على الطيران لمسافات طويلة، ليلا ونهارا، دون توقف. هذه القدرة هامة للغاية للتمكن من عبور الصحاري الكبرى الممتدة لالاف الكيلومترات بدون طعام أو ماء. قبل بدء رحلتهم لعبور الصحراء تقوم الطيور بأكل طعام غني بالدهون مثل حبوب الذرة.\n'

In [13]:
train_answers[1]

{'answer_start': 433, 'text': 'ليلا ونهارا،'}

In [14]:
answer = train_answers[1]['text']
start_idx =train_answers[1]['answer_start']
end_idx = start_idx + len(answer)
print(train_contexts[1][start_idx:end_idx])
print(answer)
print(answer == train_contexts[1][start_idx:end_idx])

ليلا ونهارا،
ليلا ونهارا،
True


In [15]:
def add_end_idx(answers, contexts):
  cnt = 0
  for answer,context in zip(answers,contexts):
    text = answer['text']
    start_idx = answer['answer_start']
    end_idx = start_idx + len(text)
    flag = True
    if context[start_idx:end_idx] == text:
      answer['answer_end'] = end_idx
      flag = False
    else:
      for i in range(1,3):
        if context[start_idx-i:end_idx-i] == text:
          answer['answer_start'] = start_idx - i
          answer['answer_end'] = end_idx - i
          flag = False
    if flag == True:
      cnt = cnt+1
  return cnt

In [16]:
error_train = add_end_idx(train_answers, train_contexts)
error_val = add_end_idx(val_answers, val_contexts)
error_test = add_end_idx(test_answers, test_contexts)
total_error = error_train +error_val + error_test
ratio = total_error/(len(train_answers)+len(val_answers)+len(test_answers))
print(f"percentage of span's error {ratio}")

percentage of span's error 0.0035920749845653028


In [18]:
from transformers import AutoTokenizer

arabert_tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv02",do_lower_case=False)

train_encodings = arabert_tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = arabert_tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [19]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [20]:
type(train_encodings['input_ids'])

list

In [24]:
train_encodings['input_ids'][0]

[2,
 17695,
 8602,
 634,
 5294,
 51029,
 2103,
 816,
 5341,
 324,
 306,
 8602,
 21208,
 307,
 33002,
 11160,
 1115,
 857,
 541,
 59021,
 201,
 13872,
 6887,
 30177,
 302,
 20,
 2797,
 450,
 8602,
 360,
 14419,
 305,
 1330,
 1411,
 54257,
 354,
 1148,
 376,
 3717,
 903,
 305,
 470,
 1251,
 20,
 669,
 14771,
 2797,
 12951,
 376,
 22787,
 310,
 305,
 10598,
 551,
 360,
 2182,
 357,
 11649,
 305,
 2733,
 103,
 2227,
 1751,
 6631,
 59755,
 3340,
 10150,
 2260,
 2797,
 360,
 2638,
 2933,
 354,
 8357,
 3007,
 3934,
 6418,
 26905,
 690,
 968,
 8602,
 20,
 669,
 14771,
 884,
 5317,
 323,
 4811,
 57673,
 2980,
 103,
 9620,
 47906,
 103,
 873,
 4913,
 20,
 450,
 5317,
 9275,
 5070,
 59029,
 306,
 14516,
 857,
 541,
 2945,
 17415,
 22649,
 350,
 51231,
 3340,
 15564,
 440,
 10984,
 20,
 600,
 2926,
 59864,
 40961,
 8304,
 2103,
 8602,
 56151,
 182,
 15564,
 22234,
 2262,
 15498,
 871,
 24784,
 24308,
 20,
 3,
 394,
 425,
 5883,
 869,
 669,
 8602,
 3235,
 1597,
 11442,
 307,
 43231,
 323,
 7929,
 1

In [23]:
print(arabert_tokenizer.decode(train_encodings['input_ids'][0]))

[CLS] هجرة الطيور هي رحلة موسمية تقوم بها أسراب من الطيور قاطعين مسافات هائلة عبر الصحاري وقمم الجبال العالية والمحيطات. تصل هذه الطيور إلى هدفها في وقت واحد يتطابق مع الوقت التي وصلت فيه في العام السابق. بعض الأنواع تصل المسافة التي تقطعها في هجرتها إلى 50 الف كيلومتر في السنة ، البعض الآخر تستمر بالطيران بدون انقطاع لمدة تصل إلى 100 ساعة مع منظومة تحديد دقيقة للاتجاهات عند تلك الطيور. بعض الأنواع لها القدرة على الطيران لمسافات طويلة ، ليلا ونهارا ، دون توقف. هذه القدرة هامة للغاية للتمكن من عبور الصحاري الكبرى الممتدة لالاف الكيلومترات بدون طعام أو ماء. قبل بدء رحلتهم لعبور الصحراء تقوم الطيور بأكل طعام غني بالدهون مثل حبوب الذرة. [SEP] ما الذي تعتمد عليه بعض الطيور الكبيرة ذات الجناحين لمساعدتها على الارتفاع ؟ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [